### Exact DMD with 4 dimentional augmented Bloch vectors for two-level spin-boson system

In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using Dates
using HDF5

In [2]:
function bloch(ρ_list)
    # Pauli matricies
    σ = [ [0 1; 1 0], [0 -im; im 0], [1 0; 0 -1], [1 0; 0 1] ]

    bloch_vec = [
        convert.(Float64, [ ρ[1,2]+ρ[2,1],
                           (ρ[1,2]-ρ[2,1])*im,
                            ρ[1,1]-ρ[2,2]       ])
                for ρ in ρ_list]
    hcat(bloch_vec...)
end

bloch (generic function with 1 method)

#### Exact DMD works as follows

Lets collect *snapshots* of the state vectors of the system $\mathbf{x}_0, \mathbf{x}_1, \dots \mathbf{x}_N $ <br>
taken at some moments of time $t_0, t_1, \dots t_n$ <br>
and collected into matrices $X$ and $X'$:

$
X = 
\begin{bmatrix}
| & | & &| \\
x_0 & x_1 & \dots & x_{n-1} \\
| & | & &| \\
\end{bmatrix}, \quad
X' = 
\begin{bmatrix}
| & | & &| \\
x_1 & x_2 & \dots & x_n \\
| & | & &| \\
\end{bmatrix}
$

from the construction:

$X' = A X$

where $A$ could be estimated as follows:
$
\hat{A}  =  X' X^{+},
$

where $X^{+}$ denotes the Moore-Penrose pseudo inverse

If consider density matrix of the two level system it could be expressed with the elements of Bloch vector:

$
\rho  = \begin{pmatrix} \rho_{00} & \rho_{01} \\ \rho_{10}  & \rho_{11}
   \end{pmatrix} = \frac{1}{2}\begin{pmatrix} 1+ b_z& b_x-i b_y \\ b_x + i b_y  & 1-b_z
   \end{pmatrix}
$

we can then use augmented four dimentional Bloch vectors as state vectors of the system :

$ \begin{pmatrix} |\\
b \\
| \\
1
\end{pmatrix} = \begin{pmatrix} b_x\\
b_y \\
b_z \\
1
\end{pmatrix} 
= \begin{pmatrix} 2 \operatorname{Re}(\rho_{01})\\
2 \operatorname{Im}(\rho_{10}) \\
\rho_{00} - \rho_{11} \\
\rho_{00} + \rho_{11} 
\end{pmatrix} 
$

we can collect evolution of Bloch vectors from the basis states  $ \ket{0},\ket{1},\ket{x},\ket{y}$ into matrices

$
Y_b = \left[
\begin{smallmatrix}
| &  & | & | & & | & | &  & | & | &  & | & | & & | \\
 b_0^{\ket{0}} & \dots & b_{n-1}^{\ket{0}} & b_0^{\ket{1}} & \dots & b_{n-1}^{\ket{1}} & b_0^{\ket{1}} & \dots & b_{n-1}^{\ket{1}} & b_0^{\ket{x}} & \dots & b_{n-1}^{\ket{x}}& b_0^{\ket{y}} & \dots & b_{n-1}^{\ket{y}} \\
| &  & | & | & & | & | &  & | & | &  & | & | & & | \\
1 &  & 1 & 1 & & 1 & 1 &  & 1 & 1 &  & 1 & 1 & & 1 \\
\end{smallmatrix} \right] \\
Y'_b = \left[
\begin{smallmatrix}
| &  & | & | & & | & | &  & | & | &  & | & | & & | \\
 b_1^{\ket{0}} & \dots & b_{n}^{\ket{0}} & b_1^{\ket{1}} & \dots & b_{n}^{\ket{1}} & b_1^{\ket{1}} & \dots & b_{n}^{\ket{1}} & b_1^{\ket{x}} & \dots & b_{n}^{\ket{x}}& b_1^{\ket{y}} & \dots & b_{n}^{\ket{y}} \\
| &  & | & | & & | & | &  & | & | &  & | & | & & | \\
1 &  & 1 & 1 & & 1 & 1 &  & 1 & 1 &  & 1 & 1 & & 1 
\end{smallmatrix} \right] 
$

Then we can find the matrix of the exact DMD liner model with the least squares:

$
  \tilde{A}^{(4D \ket{0} \ket{1} \ket{x} \ket{y})} = Y'_b Y_b^{+}, \quad \tilde{A}^{(4D \ket{0} \ket{1} \ket{x} \ket{y})}_c = \frac{\log (Y'_b Y_b^{+})}{\Delta t}
$

In [3]:
function exactDMDb4(γᵢ)

    evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

    tᵍᵉˣʸ , ρᵍᵉˣʸ  = LiPoSID.read_GEXY_timeevolution(evol_data_file_name, γᵢ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    bᵉ = bloch(ρᵉ[1:lᵐᵃˣ])
    bᵍ = bloch(ρᵍ[1:lᵐᵃˣ])
    bˣ = bloch(ρˣ[1:lᵐᵃˣ])
    bʸ = bloch(ρʸ[1:lᵐᵃˣ])

    tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
    tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 

    Δt = tᵉᶠ[2] - tᵉᶠ[1]

    Yᵉ = [bᵉ; ones(lᵐᵃˣ)']
    Yᵍ = [bᵍ; ones(lᵐᵃˣ)']
    Yˣ = [bˣ; ones(lᵐᵃˣ)']
    Yʸ = [bʸ; ones(lᵐᵃˣ)']

    Yᵉ⁻ = Yᵉ[:,1:end-1]; Yᵉ⁺ = Yᵉ[:,2:end]
    Yᵍ⁻ = Yᵍ[:,1:end-1]; Yᵍ⁺ = Yᵍ[:,2:end]
    Yˣ⁻ = Yˣ[:,1:end-1]; Yˣ⁺ = Yˣ[:,2:end]
    Yʸ⁻ = Yᵉ[:,1:end-1]; Yʸ⁺ = Yᵉ[:,2:end]

    Y⁻ = hcat(Yᵉ⁻, Yᵍ⁻, Yˣ⁻, Yʸ⁻)
    Y⁺ = hcat(Yᵉ⁺, Yᵍ⁺, Yˣ⁺, Yʸ⁺)

    A⁴ᴰ = Y⁺ * pinv(Y⁻) # Direct DMD estimation

    return A⁴ᴰ, Δt

end

exactDMDb4 (generic function with 1 method)

In [4]:
parentdir = ".."
data_dir = parentdir*"/DATA/"
println(data_dir)

models_dir = parentdir*"/MODELS/"
tests_dir = parentdir*"/TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];
basis_files = ["State_B"*string(n) for n=1:4];

all_files = vcat(dodeca_files, basis_files)
test_files = dodeca_files;

../DATA/


In [5]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

models_data_file_name = "DMD_b4_LTI_trn4_"*date_and_time_string * ".h5"

println(models_data_file_name)

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

#γ = ["0.0", "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

for γᵢ in  γ

    A⁴ᴰ, Δt = exactDMDb4(γᵢ)
    A⁴ᴰc = log(A⁴ᴰ)/Δt

    h5open(models_dir*models_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group
        γ_group["A"] = convert.(Float64, A⁴ᴰ)
        γ_group["Ac"] = convert.(Float64, A⁴ᴰc)
    end
end   

DMD_b4_LTI_trn4_2024-Sep-16_at_11-10.h5
